In [859]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
from dateutil.relativedelta import relativedelta

In [860]:
end_date_for_ind_variable = datetime.datetime(2019, 5, 18)
start_date_for_ind_variable = datetime.datetime(2016, 5, 18)
campaign_launched = datetime.datetime(2019, 6, 18)
end_date_tar_variable = datetime.datetime(2019, 12, 18)
campaign_id = 7244
number_of_campaigns = 132

# Reading Data

In [861]:
campaigns = pd.read_csv('./data/campaigns.csv')

donors = pd.read_csv('./data/donors.csv')

gifts = pd.read_csv('./data/gifts.csv')

campaigns_campaign_id = pd.read_csv('./data/selection campaign 6169.csv')

# GIFTS

### Checking data in gifts

In [862]:

print(gifts.isna().sum().sum())
for column in gifts.columns[0:]:
    print(column + " " + str(gifts[column].isna().sum()))

gifts = gifts.dropna()
gifts.info()

15637
Unnamed: 0 0
donorID 0
campaignID 15637
amount 0
date 0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 200957 entries, 1 to 216593
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  200957 non-null  int64  
 1   donorID     200957 non-null  int64  
 2   campaignID  200957 non-null  float64
 3   amount      200957 non-null  object 
 4   date        200957 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 9.2+ MB


In [863]:
gifts.drop_duplicates()

,Unnamed: 0,donorID,campaignID,amount,date
1,1,100001,1577.0,"12,39",2007-10-11
2,2,100001,1634.0,"6,69",2007-12-28
3,3,100001,1778.0,"24,79",2008-06-14
4,4,100001,1808.0,"7,44",2008-07-09
5,5,100001,1933.0,"24,79",2008-12-01
...,...,...,...,...,...
216589,216589,265584,6169.0,"300,0",2018-09-23
216590,216590,281770,6169.0,"30,0",2018-09-08
216591,216591,317880,6169.0,"157,5",2018-09-12
216592,216592,480980,6169.0,"36,0",2018-09-12


### Cleaning data in gifts

In [864]:
gifts['campaignID'] = gifts['campaignID'].fillna(0.0)
gifts['campaignID'] = gifts['campaignID'].astype(int)
gifts['amount'] = gifts['amount'].str.replace(",", ".").astype(float)
gifts['date'] = pd.to_datetime(gifts['date'])
gifts.rename(columns={'Unnamed: 0' : 'giftID'}, inplace=True)
gifts.head()

,giftID,donorID,campaignID,amount,date
1,1,100001,1577,12.39,2007-10-11
2,2,100001,1634,6.69,2007-12-28
3,3,100001,1778,24.79,2008-06-14
4,4,100001,1808,7.44,2008-07-09
5,5,100001,1933,24.79,2008-12-01


In [865]:
gifts['Quater'] = np.where(gifts['date'].dt.month < 4, 'Q1', np.where(gifts['date'].dt.month < 7, 'Q2', np.where(gifts['date'].dt.month < 10, 'Q3','Q4')))
gifts.head()

,giftID,donorID,campaignID,amount,date,Quater
1,1,100001,1577,12.39,2007-10-11,Q4
2,2,100001,1634,6.69,2007-12-28,Q4
3,3,100001,1778,24.79,2008-06-14,Q2
4,4,100001,1808,7.44,2008-07-09,Q3
5,5,100001,1933,24.79,2008-12-01,Q4


In [866]:
gifts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200957 entries, 1 to 216593
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   giftID      200957 non-null  int64         
 1   donorID     200957 non-null  int64         
 2   campaignID  200957 non-null  int32         
 3   amount      200957 non-null  float64       
 4   date        200957 non-null  datetime64[ns]
 5   Quater      200957 non-null  object        
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2), object(1)
memory usage: 10.0+ MB


# DONORS

### Checking data in donors

In [867]:
print(donors.isna().sum().sum())
for column in donors.columns[0:]:
    print(column + " " + str(donors[column].isna().sum()))
donors.head()

34
Unnamed: 0 0
donorID 0
zipcode 2
province 0
region 0
gender 32
language 0
dateOfBirth 0


,Unnamed: 0,donorID,zipcode,province,region,gender,language,dateOfBirth
0,0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967
1,1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952
2,2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986
3,3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952
4,4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986


### Cleaning data in donors

In [868]:
donors_zipcode_na = donors.loc[donors['zipcode'].isna(), :]
donors_zipcode_na.head()
donors['zipcode'] = donors['zipcode'].fillna(0)

In [869]:
donors_gender_na = donors.loc[donors['gender'].isna(), :]
donors_gender_na.head()
donors['gender'] = donors['gender'].fillna('Not Known')

In [870]:
donors.drop(['Unnamed: 0'] , axis = 1, inplace=True)
donors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44691 entries, 0 to 44690
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   donorID      44691 non-null  int64  
 1   zipcode      44691 non-null  float64
 2   province     44691 non-null  object 
 3   region       44691 non-null  object 
 4   gender       44691 non-null  object 
 5   language     44691 non-null  object 
 6   dateOfBirth  44691 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 2.4+ MB


# CAMPAIGNS

### Checking data in campaigns

In [871]:
print(campaigns.isna().sum().sum())
for column in campaigns.columns[0:]:
    print(column + " " + str(campaigns[column].isna().sum()))

campaigns.info()

0
campaignID;date;lettersSent;CostUnit 0
<class 'pandas.core.frame.DataFrame'>
Index: 367 entries, 153;13/12/2004;6 873;0 to 7536;15/12/2019;37 187;0
Data columns (total 1 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   campaignID;date;lettersSent;CostUnit  367 non-null    object
dtypes: object(1)
memory usage: 5.7+ KB


In [872]:
campaigns.head()

,campaignID;date;lettersSent;CostUnit
153;13/12/2004;6 873;0,17 €
154;01/01/2005;7 656;0,30 €
180;18/01/2005;9 933;0,20 €
433;29/01/2005;7 448;0,59 €
476;12/02/2005;6 605;0,51 €


### Cleaning data in campaigns

In [873]:
campaigns = pd.read_csv('./data/campaigns.csv')
campaigns.rename(columns={campaigns.columns[0]: "Cost" }, inplace = True)
campaigns.index.names = ['campaignID;date;lettersSent;CostUnit']
campaigns_reset = campaigns.reset_index()
campaigns_split = campaigns_reset['campaignID;date;lettersSent;CostUnit'].str.split(';', expand=True)
campaigns_split['cost'] = campaigns_reset['Cost']
campaigns_split.rename(columns={0: "campaignID" }, inplace = True)
campaigns_split.rename(columns={1: "date" }, inplace = True)
campaigns_split.rename(columns={2: "lettersSent" }, inplace = True)
campaigns_split.drop([3] , axis = 1, inplace=True)
campaigns_split.head()

,campaignID,date,lettersSent,cost
0,153,13/12/2004,6 873,17 €
1,154,01/01/2005,7 656,30 €
2,180,18/01/2005,9 933,20 €
3,433,29/01/2005,7 448,59 €
4,476,12/02/2005,6 605,51 €


In [874]:
campaigns_split['campaignID'] = campaigns_split['campaignID'].astype(int)
campaigns_split['date'] = pd.to_datetime(campaigns_split['date'])
campaigns_split['lettersSent'] = campaigns_split['lettersSent'].str.replace("\u202f", "").astype(int)
campaigns_split['cost'] = campaigns_split['cost'].str.replace("\xa0€", "").astype(int)
campaigns = campaigns_split
campaigns.head()


,campaignID,date,lettersSent,cost
0,153,2004-12-13,6873,17
1,154,2005-01-01,7656,30
2,180,2005-01-18,9933,20
3,433,2005-01-29,7448,59
4,476,2005-12-02,6605,51


In [875]:
campaigns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   campaignID   367 non-null    int32         
 1   date         367 non-null    datetime64[ns]
 2   lettersSent  367 non-null    int32         
 3   cost         367 non-null    int32         
dtypes: datetime64[ns](1), int32(3)
memory usage: 7.3 KB


### Creating Features

In [876]:
#Calculating summary for donors till Aug 4th 2018
gifts_donor_campaignID = gifts[gifts['date'] < end_date_for_ind_variable].groupby('donorID', as_index=False)['campaignID'].agg({'number_of_campaigns':'count'})
gifts_donor_total_amount = gifts[gifts['date'] < end_date_for_ind_variable].groupby('donorID', as_index=False)['amount'].agg({'total_amount':'sum'})
gifts_donor_last_date = gifts[gifts['date'] < end_date_for_ind_variable].groupby('donorID', as_index=False)['date'].agg({'last_date':'max'})
gifts_donor_last_date = gifts[gifts['date'] < end_date_for_ind_variable].groupby('donorID', as_index=False)['date'].agg({'last_date':'max'})
gifts_donor_quater_contribution = gifts[gifts['date'] < end_date_for_ind_variable].groupby(['donorID', 'Quater'], as_index=False)['date'].agg({'count'}).unstack('Quater')


donors = donors.merge(gifts_donor_campaignID, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors = donors.merge(gifts_donor_total_amount, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors = donors.merge(gifts_donor_last_date, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors = donors.merge(gifts_donor_quater_contribution, left_on = 'donorID', right_on = 'donorID', how = 'left')

donors['days_since_contributed'] = end_date_for_ind_variable - donors['last_date'] 
donors.head() 

C:\tools\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,donorID,zipcode,province,region,gender,language,dateOfBirth,number_of_campaigns,total_amount,last_date,"(count, Q1)","(count, Q2)","(count, Q3)","(count, Q4)",days_since_contributed
0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967,11.0,166.10,2011-12-30,2.0,3.0,1.0,5.0,2696 days
1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952,1.0,20.00,2015-01-10,1.0,NaN,NaN,NaN,1589 days
2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986,11.0,42.00,2019-05-06,3.0,2.0,5.0,1.0,12 days
3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952,25.0,691.43,2011-06-03,7.0,4.0,4.0,10.0,2906 days
4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986,5.0,24.96,2019-04-18,NaN,2.0,NaN,3.0,30 days


In [877]:
#Calculating summary for donors till Aug 4th 2018
gifts_donor_summary = gifts[gifts['date'] < end_date_for_ind_variable].groupby(['donorID'], as_index=False)['amount'].agg(['mean', 'min', 'max'])
gifts_donor_quater_spending = gifts[gifts['date'] < end_date_for_ind_variable].groupby(['donorID', 'Quater'], as_index=False)['amount'].agg({'mean', 'min', 'max', 'sum'}).unstack('Quater')

donors = donors.merge(gifts_donor_summary, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors = donors.merge(gifts_donor_quater_spending, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors.head()

C:\tools\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,donorID,zipcode,province,region,gender,language,dateOfBirth,number_of_campaigns,total_amount,last_date,...,"(mean, Q3)","(mean, Q4)","(min, Q1)","(min, Q2)","(min, Q3)","(min, Q4)","(sum, Q1)","(sum, Q2)","(sum, Q3)","(sum, Q4)"
0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967,11.0,166.10,2011-12-30,...,7.44,12.774,10.00,10.00,7.44,6.69,30.00,64.79,7.44,63.87
1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952,1.0,20.00,2015-01-10,...,NaN,NaN,20.00,NaN,NaN,NaN,20.00,NaN,NaN,NaN
2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986,11.0,42.00,2019-05-06,...,3.60,4.000,4.00,4.00,3.00,4.00,12.00,8.00,18.00,4.00
3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952,25.0,691.43,2011-06-03,...,24.79,28.311,24.79,24.79,24.79,24.79,194.37,114.79,99.16,283.11
4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986,5.0,24.96,2019-04-18,...,NaN,3.320,NaN,5.00,NaN,2.48,NaN,15.00,NaN,9.96


In [878]:
#Calculating age for donors till Aug 4th 2018
donors.loc[:,'age'] = donors['dateOfBirth'].apply(lambda x : end_date_for_ind_variable.year - pd.to_datetime(x).year)
donors.loc[:,'age-group'] = donors['age'] // 10 * 10
donors.head()

,donorID,zipcode,province,region,gender,language,dateOfBirth,number_of_campaigns,total_amount,last_date,...,"(min, Q1)","(min, Q2)","(min, Q3)","(min, Q4)","(sum, Q1)","(sum, Q2)","(sum, Q3)","(sum, Q4)",age,age-group
0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967,11.0,166.10,2011-12-30,...,10.00,10.00,7.44,6.69,30.00,64.79,7.44,63.87,52,50
1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952,1.0,20.00,2015-01-10,...,20.00,NaN,NaN,NaN,20.00,NaN,NaN,NaN,67,60
2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986,11.0,42.00,2019-05-06,...,4.00,4.00,3.00,4.00,12.00,8.00,18.00,4.00,33,30
3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952,25.0,691.43,2011-06-03,...,24.79,24.79,24.79,24.79,194.37,114.79,99.16,283.11,67,60
4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986,5.0,24.96,2019-04-18,...,NaN,5.00,NaN,2.48,NaN,15.00,NaN,9.96,33,30


In [879]:
gifts.head()

,giftID,donorID,campaignID,amount,date,Quater
1,1,100001,1577,12.39,2007-10-11,Q4
2,2,100001,1634,6.69,2007-12-28,Q4
3,3,100001,1778,24.79,2008-06-14,Q2
4,4,100001,1808,7.44,2008-07-09,Q3
5,5,100001,1933,24.79,2008-12-01,Q4


In [880]:
campaigns_campaign_id.nunique()

donorID    34888
dtype: int64

In [881]:
selected_donors = donors.merge(campaigns_campaign_id, left_on='donorID', right_on='donorID', how = 'inner')
selected_donors.head()
selected_donors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34874 entries, 0 to 34873
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   donorID                 34874 non-null  int64          
 1   zipcode                 34874 non-null  float64        
 2   province                34874 non-null  object         
 3   region                  34874 non-null  object         
 4   gender                  34874 non-null  object         
 5   language                34874 non-null  object         
 6   dateOfBirth             34874 non-null  object         
 7   number_of_campaigns     33802 non-null  float64        
 8   total_amount            33802 non-null  float64        
 9   last_date               33802 non-null  datetime64[ns] 
 10  (count, Q1)             18324 non-null  float64        
 11  (count, Q2)             18338 non-null  float64        
 12  (count, Q3)             14179 no

In [882]:
donors 

,donorID,zipcode,province,region,gender,language,dateOfBirth,number_of_campaigns,total_amount,last_date,...,"(min, Q1)","(min, Q2)","(min, Q3)","(min, Q4)","(sum, Q1)","(sum, Q2)","(sum, Q3)","(sum, Q4)",age,age-group
0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967,11.0,166.10,2011-12-30,...,10.00,10.00,7.44,6.69,30.00,64.79,7.44,63.87,52,50
1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952,1.0,20.00,2015-01-10,...,20.00,NaN,NaN,NaN,20.00,NaN,NaN,NaN,67,60
2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986,11.0,42.00,2019-05-06,...,4.00,4.00,3.00,4.00,12.00,8.00,18.00,4.00,33,30
3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952,25.0,691.43,2011-06-03,...,24.79,24.79,24.79,24.79,194.37,114.79,99.16,283.11,67,60
4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986,5.0,24.96,2019-04-18,...,NaN,5.00,NaN,2.48,NaN,15.00,NaN,9.96,33,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44686,144687,4900.0,Liege,Wallonia,M,FR,09/12/1970,1.0,10.00,2015-11-06,...,NaN,NaN,NaN,10.00,NaN,NaN,NaN,10.00,49,40
44687,144688,3600.0,Limburg,Flanders,M,NL,11/28/1972,6.0,43.56,2017-07-27,...,NaN,2.48,15.00,6.20,NaN,22.36,15.00,6.20,47,40
44688,144689,3061.0,Flemish Brabant,Flanders,M,NL,04/21/1967,3.0,224.79,2019-05-01,...,NaN,100.00,24.79,NaN,NaN,200.00,24.79,NaN,52,50
44689,144690,1950.0,Flemish Brabant,Flanders,M,FR,05/07/1976,1.0,40.00,2014-12-30,...,NaN,NaN,NaN,40.00,NaN,NaN,NaN,40.00,43,40


In [883]:
print(campaigns)

     campaignID       date  lettersSent  cost
0           153 2004-12-13         6873    17
1           154 2005-01-01         7656    30
2           180 2005-01-18         9933    20
3           433 2005-01-29         7448    59
4           476 2005-12-02         6605    51
..          ...        ...          ...   ...
362        7494 2019-10-17         8539    14
363        7503 2019-10-29         6126    15
364        7505 2019-11-16        29746    12
365        7519 2019-11-26        35199    36
366        7536 2019-12-15        37187    35

[367 rows x 4 columns]


In [884]:
# campaign_ids = campaigns.loc[campaigns.date > start_date_for_ind_variable, ['campaignID', 'date']]
# print(len(campaign_ids))
# campaign_ids = campaign_ids.loc[campaigns.date < , 'campaignID']
# print(len(campaign_ids))
# campaigns = pd.read_csv('./data/campaigns.csv')

column = campaigns[campaigns.campaignID == campaign_id].index[0]
row = campaigns[campaigns.campaignID == campaign_id].index[0] - number_of_campaigns
campaigns_selected = campaigns.iloc[row:column+1, :]
campaigns_selected.head()

#campaigns_selected = 7244


,campaignID,date,lettersSent,cost
290,4460,2016-05-10,24515,51
291,4500,2016-10-23,27057,25
292,4502,2016-07-11,6260,46
293,4506,2016-11-21,33275,5
294,4672,2016-03-12,7668,9


In [885]:
# Checking for mutiple donations for campaign campaign_id
gifts1 = gifts[gifts.campaignID.isin(campaigns_selected.campaignID)]
agg_gifts = gifts1.groupby(['donorID'])['campaignID'].agg(['count']).sort_values(by = 'count', ascending =False)
agg_gifts.head()


,count
donorID,
109007,38
126810,34
135651,34
109376,29
102855,28


In [886]:
gifts1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16119 entries, 13 to 216593
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   giftID      16119 non-null  int64         
 1   donorID     16119 non-null  int64         
 2   campaignID  16119 non-null  int32         
 3   amount      16119 non-null  float64       
 4   date        16119 non-null  datetime64[ns]
 5   Quater      16119 non-null  object        
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2), object(1)
memory usage: 1.3+ MB


In [887]:
selected_donors = selected_donors.merge(gifts1, left_on='donorID', right_on='donorID', how = 'inner')
# If Contribution made after May 18 

In [888]:
selected_donors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 0 to 3999
Data columns (total 41 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   donorID                 4000 non-null   int64          
 1   zipcode                 4000 non-null   float64        
 2   province                4000 non-null   object         
 3   region                  4000 non-null   object         
 4   gender                  4000 non-null   object         
 5   language                4000 non-null   object         
 6   dateOfBirth             4000 non-null   object         
 7   number_of_campaigns     3992 non-null   float64        
 8   total_amount            3992 non-null   float64        
 9   last_date               3992 non-null   datetime64[ns] 
 10  (count, Q1)             3070 non-null   float64        
 11  (count, Q2)             3040 non-null   float64        
 12  (count, Q3)             3355 non-n

In [889]:
#selected_donors['amount'].fillna(0)
#selected_donors.loc[(selected_donors['amount'] > 30) and (selected_donors['date'] > campaign_launched) and (selected_donors['date'] < end_date_tar_variable), 'has_contributed'] = 1

selected_donors['has_contributed'] = np.where((selected_donors['campaignID'] == campaign_id) & (selected_donors['amount'] > 30) & (selected_donors['date'] > campaign_launched) & (selected_donors['date'] < end_date_tar_variable),1, 0)

In [890]:
selected_donors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 0 to 3999
Data columns (total 42 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   donorID                 4000 non-null   int64          
 1   zipcode                 4000 non-null   float64        
 2   province                4000 non-null   object         
 3   region                  4000 non-null   object         
 4   gender                  4000 non-null   object         
 5   language                4000 non-null   object         
 6   dateOfBirth             4000 non-null   object         
 7   number_of_campaigns     3992 non-null   float64        
 8   total_amount            3992 non-null   float64        
 9   last_date               3992 non-null   datetime64[ns] 
 10  (count, Q1)             3070 non-null   float64        
 11  (count, Q2)             3040 non-null   float64        
 12  (count, Q3)             3355 non-n

In [891]:
for column in selected_donors[0:]:
    print(column)

donorID
zipcode
province
region
gender
language
dateOfBirth
number_of_campaigns
total_amount
last_date
('count', 'Q1')
('count', 'Q2')
('count', 'Q3')
('count', 'Q4')
days_since_contributed
mean
min
max
('max', 'Q1')
('max', 'Q2')
('max', 'Q3')
('max', 'Q4')
('mean', 'Q1')
('mean', 'Q2')
('mean', 'Q3')
('mean', 'Q4')
('min', 'Q1')
('min', 'Q2')
('min', 'Q3')
('min', 'Q4')
('sum', 'Q1')
('sum', 'Q2')
('sum', 'Q3')
('sum', 'Q4')
age
age-group
giftID
campaignID
amount
date
Quater
has_contributed


In [892]:
drop_columns = ['zipcode', 'dateOfBirth', 'last_date', 'giftID', 'campaignID', 'Quater', 'date']
selected_donors = selected_donors.drop(drop_columns, axis = 1)


In [893]:
selected_donors = selected_donors.rename(columns={('count', 'Q1') : 'Q1_Donations', ('count', 'Q2') : 'Q2_Donations', ('count', 'Q3') : 'Q3_Donations', ('count', 'Q4') : 'Q4_Donations'})


In [894]:
selected_donors = selected_donors.rename(columns={('mean', 'Q1') : 'Q1_Mean', ('mean', 'Q2') : 'Q2_Mean', ('mean', 'Q3') : 'Q3_Mean', ('mean', 'Q4') : 'Q4_Mean'})

In [895]:
selected_donors = selected_donors.rename(columns={('min', 'Q1') : 'Q1_Min', ('min', 'Q2') : 'Q2_Min', ('min', 'Q3') : 'Q3_Min', ('min', 'Q4') : 'Q4_Min'})

In [896]:
selected_donors = selected_donors.rename(columns={('max', 'Q1') : 'Q1_Max', ('max', 'Q2') : 'Q2_Max', ('max', 'Q3') : 'Q3_Max', ('max', 'Q4') : 'Q4_Max'})

In [897]:
selected_donors = selected_donors.rename(columns={('sum', 'Q1') : 'Q1_Amount', ('sum', 'Q2') : 'Q2_Amount', ('sum', 'Q3') : 'Q3_Amount', ('sum', 'Q4') : 'Q4_Amount'})

In [898]:
selected_donors['days_since_contributed'] = selected_donors['days_since_contributed'].dt.days

In [899]:
for column in selected_donors[0:]:
    print(column)

donorID
province
region
gender
language
number_of_campaigns
total_amount
Q1_Donations
Q2_Donations
Q3_Donations
Q4_Donations
days_since_contributed
mean
min
max
Q1_Max
Q2_Max
Q3_Max
Q4_Max
Q1_Mean
Q2_Mean
Q3_Mean
Q4_Mean
Q1_Min
Q2_Min
Q3_Min
Q4_Min
Q1_Amount
Q2_Amount
Q3_Amount
Q4_Amount
age
age-group
amount
has_contributed


In [900]:
selected_donors.loc[selected_donors.has_contributed == 1, 'has_contributed'].sum()

184

In [901]:
selected_donors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 0 to 3999
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   donorID                 4000 non-null   int64  
 1   province                4000 non-null   object 
 2   region                  4000 non-null   object 
 3   gender                  4000 non-null   object 
 4   language                4000 non-null   object 
 5   number_of_campaigns     3992 non-null   float64
 6   total_amount            3992 non-null   float64
 7   Q1_Donations            3070 non-null   float64
 8   Q2_Donations            3040 non-null   float64
 9   Q3_Donations            3355 non-null   float64
 10  Q4_Donations            2770 non-null   float64
 11  days_since_contributed  3992 non-null   float64
 12  mean                    3992 non-null   float64
 13  min                     3992 non-null   float64
 14  max                     3992 non-null   

In [902]:
selected_donors.loc[selected_donors.has_contributed == 1, 'has_contributed'].sum()

184

In [903]:
selected_donors.to_csv(os.path.join('./data',r'processed_table_7244.csv'))